In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install fiftyone==0.18

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
import fiftyone as fo
from fiftyone import ViewField as F

Migrating database to v0.18.0


INFO:fiftyone.migrations.runner:Migrating database to v0.18.0


In [ ]:
import json
import pickle
import numpy as np

In [ ]:
# import pycocotools
from pycocotools import mask as mask_utils

## Convert detections

In [ ]:
# file .pkl with prediction
#detection_file = '/content/drive/MyDrive/granum/results.pkl'

In [ ]:
#!python /content/drive/MyDrive/granum/04_tools/extract_dets_from_mmdet.py $detection_file

# Visualization

In [ ]:
images_path = '/content/drive/MyDrive/granum/02_dane/zbior_T_JPEG'
annotations_path = '/content/drive/MyDrive/granum/ann_T_empty4.json'
detections_path = '/content/drive/MyDrive/granum/results/015_20230405_drnew/results.json'

In [ ]:
dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path=images_path,
    labels_path=annotations_path,
    include_id=True,
    label_field='ground_truth',
    label_types='segmentations', #'detections' #'segmentations'
)

 100% |███████████████████| 39/39 [39.4ms elapsed, 0s remaining, 990.9 samples/s]     


INFO:eta.core.utils: 100% |███████████████████| 39/39 [39.4ms elapsed, 0s remaining, 990.9 samples/s]     


In [ ]:
len(dataset)

39

In [ ]:
dataset.default_classes = [x['name'] for x in json.load(open(annotations_path))['categories']]

In [ ]:
predictions = json.load(open(detections_path))

In [ ]:
len(predictions)

39

In [ ]:
class_names = dataset.default_classes

In [ ]:
print(dataset)

Name:        2023.06.13.12.01.31
Media type:  image
Num samples: 39
Persistent:  False
Tags:        []
Sample fields:
    id:                   fiftyone.core.fields.ObjectIdField
    filepath:             fiftyone.core.fields.StringField
    tags:                 fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:             fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth_coco_id: fiftyone.core.fields.IntField


In [ ]:
assert len(dataset) == len(predictions)
for i_sample, sample in enumerate(dataset.iter_samples()):
    preds = predictions[i_sample]

    w = sample.metadata.width
    h = sample.metadata.height

    detections = []
    for pred in preds:
        x1, y1, x2, y2 = [int(x) for x in pred['bounding_box']]
        rel_box = [x1 / w, y1 / h, (x2 - x1) / w, (y2 - y1) / h]

#         pred['mask']['counts'] = pred['mask']['counts'].encode()
        mask = mask_utils.decode(pred["mask"])
        mask_box = mask[y1:y2, x1:x2]

        detections.append(
            fo.Detection(
                label=class_names[pred["label"]],
                bounding_box=rel_box,
                confidence=pred["confidence"],
                mask=mask_box
            )
        )

    sample['predictions'] = fo.Detections(detections=detections)
    sample.save()


In [ ]:
half_conf_view = dataset.filter_labels("predictions", F("confidence") > 0.5)

In [ ]:
len(half_conf_view)

38

In [ ]:
session = fo.launch_app(half_conf_view, port=5000) #fo.launch_app(dataset)

# Nowa sekcja